In [1]:
using Random
using LinearAlgebra

# Define the PSO structure
mutable struct Particle
    position::Vector{Float64}
    velocity::Vector{Float64}
    pbest::Vector{Float64}
    pbest_val::Float64
end

# PSO function
function pso(f, n_particles::Int, n_dims::Int, max_iter::Int, ω::Float64, c1::Float64, c2::Float64)
    particles = Particle[]
    global_best = zeros(Float64, n_dims)
    global_best_val = Inf

    # Initialize particles
    for _ in 1:n_particles
        position = randn(n_dims)
        velocity = zeros(n_dims)
        pbest = copy(position)
        pbest_val = f(position)
        push!(particles, Particle(position, velocity, pbest, pbest_val))
        
        if pbest_val < global_best_val
            global_best[:] = pbest
            global_best_val = pbest_val
        end
    end

    # PSO iterations
    for iter in 1:max_iter
        for particle in particles
            # Update velocity
            particle.velocity .= ω .* particle.velocity .+
                                   c1 .* rand().*(particle.pbest .- particle.position) .+
                                   c2 .* rand().*(global_best .- particle.position)
            
            # Update position
            particle.position .+= particle.velocity
            
            # Evaluate fitness
            current_val = f(particle.position)
            
            # Update personal best
            if current_val < particle.pbest_val
                particle.pbest = copy(particle.position)  # Create a new Particle object
                particle.pbest_val = current_val
                
                # Update global best
                if current_val < global_best_val
                    global_best[:] = particle.position
                    global_best_val = current_val
                end
            end
        end
    end
    
    return global_best, global_best_val
end

# Gradient Descent function
function gradient_descent(f, gradient, initial_position::Vector{Float64}, learning_rate::Float64, max_iter::Int)
    current_position = copy(initial_position)
    
    for iter in 1:max_iter
        grad = gradient(current_position)
        current_position .-= learning_rate .* grad
        
        # Check convergence
        if norm(grad) < 1e-6
            break
        end
    end
    
    return current_position, f(current_position)
end

# Hybrid PSO with Gradient Descent
function hybrid_pso_with_gradient_descent(f, gradient, n_particles::Int, n_dims::Int, max_iter::Int, ω::Float64, c1::Float64, c2::Float64, learning_rate::Float64)
    # Run PSO to get initial solution
    initial_solution, _ = pso(f, n_particles, n_dims, max_iter, ω, c1, c2)
    
    # Run gradient descent starting from the PSO solution
    final_solution, final_value = gradient_descent(f, gradient, initial_solution, learning_rate, max_iter)
    
    return final_solution, final_value
end

# Example usage
f(x) = sum(x.^2) # Objective function: Sphere function
gradient(x) = 2*x # Gradient of the Sphere function
n_particles = 10
n_dims = 2
max_iter = 100
ω = 0.7
c1 = 1.5
c2 = 1.5
learning_rate = 0.1

solution, value = hybrid_pso_with_gradient_descent(f, gradient, n_particles, n_dims, max_iter, ω, c1, c2, learning_rate)
println("Optimal Solution: ", solution)
println("Optimal Value: ", value)


Optimal Solution: [1.559236958244125e-7, -2.898591291388931e-7]
Optimal Value: 1.0833051366470144e-13
